In [20]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import spacy
NER = spacy.load("en_core_web_sm")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

df = pd.read_csv('amazon_alexa.tsv', sep='\t')

df.head()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sicom\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sicom\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\sicom\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [21]:
df = df.drop(['rating', 'date','feedback' ,'variation'], axis=1)

In [22]:

df['verified_reviews_POS'] = df['verified_reviews'].apply(lambda x: word_tokenize(x))
df

,verified_reviews,verified_reviews_POS
0,Love my Echo!,"[Love, my, Echo, !]"
1,Loved it!,"[Loved, it, !]"
2,"Sometimes while playing a game, you can answer...","[Sometimes, while, playing, a, game, ,, you, c..."
3,I have had a lot of fun with this thing. My 4 ...,"[I, have, had, a, lot, of, fun, with, this, th..."
4,Music,[Music]
...,...,...
3145,"Perfect for kids, adults and everyone in betwe...","[Perfect, for, kids, ,, adults, and, everyone,..."
3146,"Listening to music, searching locations, check...","[Listening, to, music, ,, searching, locations..."
3147,"I do love these things, i have them running my...","[I, do, love, these, things, ,, i, have, them,..."
3148,Only complaint I have is that the sound qualit...,"[Only, complaint, I, have, is, that, the, soun..."


In [23]:
df['verified_reviews_POS'] = df['verified_reviews_POS'].apply(lambda x: nltk.pos_tag(x))
df


,verified_reviews,verified_reviews_POS
0,Love my Echo!,"[(Love, VB), (my, PRP$), (Echo, NN), (!, .)]"
1,Loved it!,"[(Loved, VBN), (it, PRP), (!, .)]"
2,"Sometimes while playing a game, you can answer...","[(Sometimes, RB), (while, IN), (playing, VBG),..."
3,I have had a lot of fun with this thing. My 4 ...,"[(I, PRP), (have, VBP), (had, VBD), (a, DT), (..."
4,Music,"[(Music, JJ)]"
...,...,...
3145,"Perfect for kids, adults and everyone in betwe...","[(Perfect, NNP), (for, IN), (kids, NNS), (,, ,..."
3146,"Listening to music, searching locations, check...","[(Listening, VBG), (to, TO), (music, NN), (,, ..."
3147,"I do love these things, i have them running my...","[(I, PRP), (do, VBP), (love, VB), (these, DT),..."
3148,Only complaint I have is that the sound qualit...,"[(Only, RB), (complaint, NN), (I, PRP), (have,..."


In [24]:
dictionary = {}

In [25]:
for rev in df['verified_reviews_POS']:
    for word in rev:
        if word[1] in dictionary:
            #if word[0] not in word[1]:
            dictionary[word[1]].add(word[0])
        else:
            dictionary[word[1]] = set(word[0])


In [26]:
for key in dictionary:
    dictionary[key] = list(dictionary[key])

In [27]:
print(dictionary)

{'VB': ['end', '....', 'tinny', 'toy', 'whisper', 'hope', 'introduce', 'be', 'avoid', 'prefer', 'google', 'Yes', 'test', 'though', 'punch', 'maintain', 'speak', 'change', 'wall', 'repeat', 'review', 'ask', 'alone', 'song', 'majes', 'compare', 'answer', 'select', 'reboot', 'Overall', 'Otherwise', 'imagine', 'stick', 'drop', 'self', 'get', 'deal', 'Return', 'display', 'silly', 'buy.We', 'override', 'rub', 'voice', 'Luv', 'A+++++', 'Call', 'tell', 'play', 'task', 'please', 'decrease', 'take', 'sync', 'perform', 'fast', '‘', 'rent', 'afford', 'rest', 'update', '..', 'Bluetooth', 'shuffle', 'back.The', 'discourage', 'simplify', 'maneuver', 'Say', 'choose', 'remember', 'hook', 'sink', 'engage', 'dial', 'complain.I', 'live', 'expect', 'eliminate', 'offer', 'follow', 'mean', 'fit', 'skill', 'default', 'register', 'find', 'load', 'Do', 'clean', 'youtube', 'throw', 'sell', 'look', 'spend', 'sign', 'e', 'Make', 'exchange', 'split', 'install', 'stuff', 'resist', 'shout', 'worry', 'ise', 'replace',

In [28]:
out = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dictionary.items() ]))
out

,VB,PRP$,NN,.,VBN,PRP,RB,IN,VBG,DT,...,RBS,#,EX,FW,``,'',UH,NNPS,WP$,SYM
0,end,his,defuser.I,!,bought,herself,so,....,summoning,bluetooth,...,t,#,T,’,``,'',Awesome,A,w,t
1,....,Its,speech,?,worked,We,Up,bc,frustrating,No,...,m,NaN,e,quedó,`,',listen,e,e,b
2,tinny,its,bug,.,reduced,It,user,so,falling,both,...,most,NaN,weather,rapidez,NaN,NaN,Well,Rivers,h,..
3,toy,Our,decent,NaN,gone,i,continually,........,finding,this,...,s,NaN,h,etc,NaN,NaN,e,Answers,s,m
4,whisper,My,finding,NaN,recorded,They,yr,along,Acting,all,...,o,NaN,There,c,NaN,NaN,No,c,o,o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,NaN,NaN,plenty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1677,NaN,NaN,blast,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1678,NaN,NaN,scent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1679,NaN,NaN,individual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
!pip install xlwt
out.to_excel('postagging.xlsx')

In [30]:
df['verified_reviews_NER'] = df['verified_reviews'].apply(lambda x: NER(x))

In [31]:
df['verified_reviews_NER']

0                                     (Love, my, Echo, !)
1                                          (Loved, it, !)
2       (Sometimes, while, playing, a, game, ,, you, c...
3       (I, have, had, a, lot, of, fun, with, this, th...
4                                                 (Music)
                              ...                        
3145    (Perfect, for, kids, ,, adults, and, everyone,...
3146    (Listening, to, music, ,, searching, locations...
3147    (I, do, love, these, things, ,, i, have, them,...
3148    (Only, complaint, I, have, is, that, the, soun...
3149                                               (Good)
Name: verified_reviews_NER, Length: 3150, dtype: object

In [32]:
dict_ner = {}
for rev in df['verified_reviews_NER']:
    for word in rev.ents:
        if word.label_ in dict_ner:
            dict_ner[word.label_].add(word.text)
        else:
            dict_ner[word.label_] = set(word.text)
            
dict_ner

        

{'ORG': {'3&#34',
  '3rd Echo',
  '5GHz',
  'A',
  'A+++++',
  "A.I.'s",
  'ABC',
  'AI',
  'ALEXA',
  'ALLRecipes',
  'AMAZON',
  'ASK',
  'Ac',
  'Affirm.com',
  'Alexa',
  'Alexa/Echo/Computer',
  'Amazon',
  'Amazon Alexa',
  'Amazon Echo',
  'Amazon Fire TV',
  'Amazon Fire Tv',
  'Amazon Music',
  'Amazon Prime Music',
  'Amazon Prime Video',
  'Amazon Video',
  'AnyList',
  'Anypod',
  'Apple',
  'Apple Music',
  'Apple TV',
  'Apple’s Siri',
  'Apps',
  'Arlo',
  'Arlo Security',
  'Audible & Kindle & Alex',
  'Awesome',
  'BFF',
  'BOSE',
  'Beautiful Music',
  'Better',
  'Bloomberg TV',
  'British Accent',
  'CBS',
  'CNN',
  'CSI',
  'Chromecast',
  'Coop',
  'DOES',
  'DOT',
  'Denon AV',
  'DirectTV Now',
  'Directv',
  'Dot',
  'Dots and Spot',
  'Drop In',
  'ECHO',
  'EEAANH EEAANH EEAANH',
  'EQ',
  'Easy & fun',
  'Echo',
  'Echo Buttons',
  'Echo Dot',
  'Echo Dots & am',
  'Echo Smart',
  'Echo Spot',
  'Echo Spot - White',
  'Echos',
  'Exho',
  'FIRE',
  'FW',
  

In [33]:
for key in dict_ner:
    dict_ner[key] = list(dict_ner[key])

In [34]:
out_ner = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in dict_ner.items() ]))
out_ner

,ORG,CARDINAL,ORDINAL,DATE,WORK_OF_ART,LOC,GPE,TIME,MONEY,PERSON,PRODUCT,FAC,NORP,EVENT,LANGUAGE,PERCENT,QUANTITY,LAW
0,Apple Music,229,2nd,May,LOVE MY ECHO SHOW,echo,echo,6 hours,34;lights.&#34,Aunt,Kasa,Echo Tower,an Echo Dot,This Echo Dot,English,1,,e
1,REFURBISHED ECHO DOT &,hundreds,First,10 year old,L,the Echo Show,Alexas,two second,34;turn,Phillips,SHOW,,Southern,The Echo Dot,E,15%,b,the Training Coordinator
2,the PhilipsHue Light Bulb,a million,3xs,the 1st,The Echo Plus,Dots,Netflix,one night,29.99,Contacted Amazon,b,x,Americans,an Echo Dot,i,around 20%,1GB,
3,Sprint,hundred,Third,a year,Mother’s Day,the Echo Dots,i,4 hours,99,Hue,i,E,i,c,h,%,i,o
4,Phillips,2package,6th,August,an Echo Dot,Long Island,f,4am,100,Alexa Echo,Spot,c,I,,Spanish,0,m,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,CSI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,AI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199,the Amazon Echo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
200,Amazon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
out_ner.to_excel('ner.xlsx')

In [36]:
dict_dep = { 'Token': [], 'Relation':[],'Head':[],'Children':[]}
            

In [37]:
for rev in df['verified_reviews_NER']:
    for token in rev:
        dict_dep['Token'].append(str(token.text))
        dict_dep['Relation'].append(str(token.dep_))
        dict_dep['Head'].append(str(token.head.text))
        dict_dep['Children'].append(str([child for child in token.children]))

            
out_dep = pd.DataFrame(dict_dep)
out_dep.head(-10)

,Token,Relation,Head,Children
0,Love,ROOT,Love,"[Echo, !]"
1,my,poss,Echo,[]
2,Echo,dobj,Love,[my]
3,!,punct,Love,[]
4,Loved,ROOT,Loved,"[it, !]"
...,...,...,...,...
92990,echo,pobj,with,[size]
92991,or,cc,go,[]
92992,make,conj,go,[sure]
92993,sure,ccomp,make,[hook]


In [38]:
out_dep.to_excel('dep.xlsx')